In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
res_dir <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc'
phewas_hits <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.tsv')
phewas_hits_ld_gwas_catalog_summary <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog.summary.tsv')


In [3]:
df <- fread(phewas_hits)

#### import the manual annotations from google drive

In [4]:
library(googlesheets)
gs_auth(token = "/home/users/ytanigaw/.googlesheets_token.rds")

phewas_hits_ld_gwas_catalog_gdrive <- 
'https://docs.google.com/spreadsheets/d/1yocReg2dL84x1NQDEyc9ywEDJM6gdfr2ELNlCcP3zPo' %>% 
gs_url() %>% 
gs_read(ws = 'ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog') %>%
rename('CHROM' = '#CHROM')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1yocReg2dL84x1NQDEyc9ywEDJM6gdfr2ELNlCcP3zPo
Sheet successfully identified: "phewas"
Accessing worksheet titled 'ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog'.
Parsed with column specification:
cols(
  .default = col_character(),
  `#CHROM` = col_double(),
  POS = col_double(),
  OBS_CT = col_double(),
  OR = col_double(),
  `LOG(OR)_SE` = col_double(),
  Z_STAT = col_double(),
  P = col_double(),
  LD_CHROM = col_double(),
  LD_POS = col_double(),
  LD_R2 = col_double(),
  phewas_idx = col_double(),
  EBI_P = col_double(),
  is_novel = col_logical()
)
See spec(...) for full column specifications.


let's check if the PheWAS results are annotated for both novel and NOT novel.

In [5]:
phewas_hits_ld_gwas_catalog_gdrive %>% 
select(CHROM, POS, ID, GBE_ID, is_novel) %>% 
unique() %>%
group_by(CHROM, POS, ID, GBE_ID) %>%
summarise(n_flag = n()) %>%
filter(n_flag > 1)

CHROM,POS,ID,GBE_ID,n_flag
<dbl>,<dbl>,<chr>,<chr>,<int>


the manual annotations are consistent

#### number of association counts

In [6]:
phewas_hits_ld_gwas_catalog_gdrive %>%  
select(ID, GBE_ID, is_novel) %>% 
unique() %>%
count(is_novel)


is_novel,n
<lgl>,<int>
FALSE,97
TRUE,170


#### `GBE_ID` to GBE short name

In [7]:

GBE_names_df <- 'https://docs.google.com/spreadsheets/d/1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A' %>% 
gs_url() %>% gs_read(ws = 'GBE_names')


Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Sheet successfully identified: "GBE_names"
Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_name_len = col_double()
)


#### make a summary table


In [8]:
imp_nc_res_summary_df <- df %>% 
left_join(
    GBE_names_df %>%
    select(GBE_ID, GBE_short_name),
    by='GBE_ID'
) %>%
left_join(
    phewas_hits_ld_gwas_catalog_gdrive %>%  
    select(ID, GBE_ID, is_novel) %>% 
    unique(),
    by=c('ID', 'GBE_ID')
)


In [9]:
imp_nc_res_summary_df %>% 
rename('#CHROM' = 'CHROM') %>%
fwrite(phewas_hits_ld_gwas_catalog_summary, sep='\t')


In [10]:
phewas_hits_ld_gwas_catalog_summary

[1] "/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc/ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog.summary.tsv"